In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyodesys.symbolic import SymbolicSys
%matplotlib inline

In [ ]:
sys1 = SymbolicSys.from_callback(lambda t, y, p: [-y[0]/t], 1)
sys1.exprs

In [ ]:
def vary(sys, kws):
    res = [sys.integrate((1, 1e20), [1.0], atol=1e-50, nsteps=64000, **kw) for kw in kws]
    return res

In [ ]:
kws = [
    dict(integrator='cvode', method='bdf', rtol=1e-6),
    dict(integrator='cvode', method='adams', rtol=1e-10),
    dict(integrator='gsl', method='bsimp', rtol=1e-10),
    dict(integrator='gsl', method='msbdf', rtol=1e-10)
]

res1 = vary(sys1, kws)

In [ ]:
def plot_variation(res, kws):
    fig, axes = plt.subplots(1, len(res), figsize=(16,6), dpi=150)
    for ax, r, kw in zip(axes, res, kws):
        #r.plot(ax=ax)
        rx = 1/r.xout
        ax.plot(r.xout, np.abs(r.yout.squeeze() - rx)/rx, label='observed')
        ax.set_xscale('log')
        ax.set_yscale('linear')
        ax.set_ylabel('|relative error|')

        random_walk = np.sqrt(1.0+np.array(range(r.xout.size)))  # non-systematic errors (no bias)
        #ax.plot(r.xout, random_walk*kw['rtol'], label='prognosis')

        ax.legend()

        ax2 = ax.twinx()
        ax2.plot(r.xout[:-1], np.diff(r.xout), color='k', linewidth=0.5
                 #, ls='None', marker='.', markersize=0.5, alpha=0.3
                )
        ax2.set_yscale('log')
        ax2.set_ylim([1e-12, 1e20])
        ax2.set_ylabel('step size')

        ax.set_title(f"{kw['integrator']}\n{kw['method']}\n{r.info['n_steps']=}\n{r.info['time_cpu']=}\n ")

    fig.tight_layout()

In [ ]:
plot_variation(res1, kws)

In [ ]:
sys2 = sys1.as_autonomous()
sys2.exprs

In [ ]:
plot_variation(vary(sys2, kws), kws)